# [해커톤3] 구글 리뷰 크롤링하기

> * urllib는 파이썬에서 인터넷에서 데이터를 받아 오는 기능들이 있는 패키지로 기본적으로 내장되어 있음
> * urllib에 인터넷 주소URL을 넣고 실행하면 데이터를 텍스트 형태로 받아올 수 있음 : 크롤링
> * BeautifulSoup은 데이터를 추출하는데 필요한 기능이 들어 있는 라이브러리
> * BeautifulSoup은 크롤링한 데이터에서 필요한 내용만 추출하는 라이브러리 : 파싱

## 구글지도 리뷰 크롤링

In [1]:
import urllib
import urllib.request
from urllib.request import urlopen
import selenium 
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd
import os

In [2]:
link = 'https://www.google.co.kr/maps/place/%EC%88%98%ED%86%B5%EA%B3%A8%EC%9C%A0%EC%9B%90%EC%A7%80/@36.3462199,127.2871404,16z/data=!4m11!1m2!2m1!1z7IiY7Ya16rOo!3m7!1s0x357ab320dafefd2b:0xe6383df46636c16c!8m2!3d36.3423148!4d127.28525!9m1!1b1!15sCgnsiJjthrXqs6iSARJ0b3VyaXN0X2F0dHJhY3Rpb24?hl=ko'
driver = webdriver.Chrome('./chromedriver')
driver.get(link)                 # 자동으로 해당 링크를 연다
os.makedirs('result', exist_ok = True)   # 결과 저장 디렉토리


### 자동 스크롤 다운 하기

스크래핑하기 전에 대부분의 다른 최신 웹 사이트와 마찬가지로 페이지가 AJAX(동적페이지)를 사용하여 구현되므로 모든 리뷰를 아래로 스크롤하여 모든 리뷰를 로드해야한다.

In [ ]:
# scroll_cnt = 10

# for i in range(scroll_cnt):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)

#     # click 'load more':
#     try:
#         load_more = driver.find_elements_by_class_name("section-expand-review")
#         for i in load_more:
#             i.click()
#     except:
#         pass

### 리뷰정보 가져오기

In [4]:
# 리뷰 가져오기
elem = driver.find_element_by_class_name('widget-pane-content-holder')
reviews = elem.find_elements_by_xpath('//*[@class="section-review NIyLF-haAclf gm2-body-2"]//div[@class="ODSEW-ShBeI-content"]')
print('there are %d reviews avaliable!' % len(reviews))
print('writing the data...')

there are 543 reviews avaliable!
writing the data...


In [5]:
# 해당 정보 저장하기

df = pd.DataFrame(columns=['name', 'ratings', 'date', 'comment'])

In [6]:
for review in reviews:
    soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')
    
    name = soup.find(class_= "section-review-title").text
    
    ratings = int(soup.find('span', class_='section-review-stars').get('aria-label').replace('별표', '').replace('개','').strip())
    
    date= soup.find(class_= "section-review-publish-date").text
     
    comment = soup.find('span', class_='section-review-text').text
    if not comment:
        comment = "없음" 
        
    df = df.append({
        'name': name,
        'ratings': ratings,
        'date' : date,
        'comment' : comment
    }, ignore_index=True)


In [7]:
filename = datetime.now().strftime('result/%Y-%m-%d_%H-%M-%S.csv')
df.to_csv(filename, encoding='utf-8-sig', index=False)
driver.stop_client()
driver.close()

print('끝!')

끝!


## 결과 확인

In [ ]:
import os
from os.path import join

In [ ]:
data_dir = os.getenv('HOME')+'/result'

place_path = join(data_dir, '뿌리공원.csv')
place_check = pd.read_csv(place_path)

place_check.head(20)

참고
http://egloos.zum.com/mcchae/v/11281390

## [참고] 요소찾기

id_r = r.find('button', 
              class_='section-review-action-menu')['data-review-id']
username = r.find('div', 
                  class_='section-review-title').find('span').text
try:
    review_text = r.find('span', class_='section-review-text').text
except Exception:
    review_text = None
rating = r.find('span', class_='section-review-stars')['aria-label']
rel_date = r.find('span', class_='section-review-publish-date').text